<a href="https://colab.research.google.com/github/abdallaho98/AutoSpeech/blob/master/NB_and_Logistical_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 138kB/s 
     |████████████████████████████████| 421.8MB 39kB/s 
     |████████████████████████████████| 983kB 32.8MB/s 
     |████████████████████████████████| 245kB 46.9MB/s 
     |████████████████████████████████| 778kB 41.1MB/s 
     |████████████████████████████████| 471kB 36.1MB/s 
     |████████████████████████████████| 3.9MB 36.4MB/s 
     |████████████████████████████████| 450kB 43.5MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 1.1MB 37.8MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 
     |████████████████████████████████| 3.0MB 38.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.20.0-cp36-none-any.whl size=25269082 sha256=9c3d1820623f75fab85be16597fc7dd66ecb29f9a6f5b8de1d6efc4ffd6bd9f2
  Stored in directory: /root/.cache/pip/wheels/0d/90/b9/7c4d11547b37c382e59d98c3587c53364971a4547b913fa9b1
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [3]:
import ktrain
from ktrain import text

In [ ]:
import pandas as pd
filepath = 'legalts.txt'
df = pd.DataFrame(columns=('rating', 'review'))
with open(filepath) as fp:
   line = fp.readline()
   cnt = 0
   while line:
       x = line.strip().split("  ")
       df.loc[cnt] = [int(x[1]) , x[0]]
       line = fp.readline()
       cnt += 1
df['rating'] = df['rating'].apply(lambda x: x)
df.head()

,rating,review
0,4,ليس للشركاء في مال شائع أن يطلبوا قسمته إذا تب...
1,5,إذا وقع تحريف في نص السفتجة فالأشخاص الموقعون ...
2,2,يجوز الحجز على الأجر أو المرتب بقيمة النفقة ال...
3,3,يعين القاضي طبقا لأحكام قانون الأسرة، مقدما من...
4,2,يجوز للعموم المعارضة في أي تصحيح قضائي أو إدار...


In [5]:
import pandas as pd
import re
import string
import snowballstemmer
ar_light_stem = snowballstemmer.stemmer('arabic') 
filepath = 'legalts.txt'
df = pd.DataFrame(columns=('rating', 'review'))
with open(filepath) as fp:
   line = fp.readline()
   cnt = 0
   while line:
       x = line.strip().split("  ")
       result = re.sub(r'[0-9,.()،]+', '', x[0])
       listStrin = [ar_light_stem.stemWord(text) for text in result.split(' ')]
       strin = ' '.join(listStrin)
       df.loc[cnt] = [int(x[1]) , strin]
       line = fp.readline()
       cnt += 1
df['rating'] = df['rating'].apply(lambda x: x)
df.head()

,rating,review
0,4,ليس شركاء في مال شايع ان يطلب قسم اذا تبين من ...
1,5,اذا وقع تحريف في نص سفتج فالاشخاص موقع علي بعد...
2,2,يجوز حجز علي اجر او مرتب قيم نفق غذاء اذا كان ...
3,3,يعين قاض طبق لاحكام قانون اسر مقدم من بين اقار...
4,2,يجوز عموم معارض في اي تصحيح قضاء او ادار لعقد ...


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['review'].values, 
                 df['rating'].values,        
                 test_size=0.2)

In [ ]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
vect = CountVectorizer(tokenizer=tokenize)
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [ ]:
model = LogisticRegression(C=3, dual=False)
model.fit(tf_train, y_train)
preds = model.predict(tf_test)
accuracy = (preds == y_test).mean()
print(accuracy)

0.7874396135265701


In [ ]:
vect = TfidfVectorizer(strip_accents='unicode', tokenizer=tokenize, ngram_range=(1, 3), max_df=0.9, min_df=5, sublinear_tf=True)
tfidf_train = vect.fit_transform(X_train)
tfidf_test = vect.transform(X_test)
model = LogisticRegression(C=30,max_iter=500, dual=False)
model.fit(tfidf_train, y_train)
preds = model.predict(tfidf_test)
accuracy = (preds==y_test).mean()
print(accuracy)

0.7681159420289855


In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB().fit(tfidf_train.todense(), y_train)
predicted_y = model.predict(tfidf_test.todense())
from sklearn.metrics import accuracy_score

# now calculating that how much accurate our model is with comparing our predicted values and y_test values
accuracy_score = accuracy_score(y_test, predicted_y) 
print (accuracy_score)

0.46859903381642515


In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(tfidf_train, y_train)
mnb.score(tfidf_test, y_test)

0.5748792270531401

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(tf_train, y_train)
mnb.score(tf_test, y_test)

0.5845410628019324

In [30]:
import gensim
from sklearn.preprocessing import LabelEncoder
w2v = gensim.models.Word2Vec(list(df['review'].values), size=350, window=10, min_count=2, iter=20)
def document_vector(doc):
    doc = [word for word in doc if word in w2v.wv.vocab]
    return np.mean(w2v[doc], axis=0)
datadata = df['review'][:900].apply(document_vector)
testtest = df['review'][900:1000].apply(document_vector)
lb = LabelEncoder()
y = lb.fit_transform(df['rating'][:900].values)
X = list(datadata)
X_test = list(testtest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [48]:
clf = LogisticRegression(C=10,max_iter=500, dual=False)
clf.fit(X, y)
y_test = clf.predict(X_test)
y_pred = lb.inverse_transform(y_test)
ytester = lb.fit_transform(df['rating'][900:1000].values)
from sklearn.metrics import accuracy_score
print('Testing accuracy LG %s' % accuracy_score(y_test, ytester))
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X, y)
mnb.score(y_pred, ytester)

Testing accuracy LG 0.35


ValueError: ignored